In [1]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117


In [2]:
pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 619 kB 73.2 MB/s 
     |████████████████████████████████| 145 kB 82.7 MB/s 
     |████████████████████████████████| 47.6 MB 108.4 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.6.0.66
    Uninstalling opencv-python-headless-4.6.0.66:
      Successfully uninstalled opencv-python-headless-4.6.0.66


In [3]:
import torch

In [4]:
# Checking if the CUDA activated in the environment
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
Tesla T4


In [5]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
from matplotlib import pyplot as pit
from PIL import ImageFont, ImageDraw, Image, ImageGrab
from datetime import datetime
import pytz
import cv2
import numpy as np
import pandas as pd
import torch
import easyocr
import io
import html
import time

In [6]:
tz_dhaka = pytz.timezone('Asia/Dhaka')
datetime_dhaka = datetime.now(tz_dhaka)
DATE = datetime_dhaka.strftime("%m-%d-%Y")
TIME = datetime_dhaka.strftime("%H:%M:%S")

In [7]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [8]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [9]:
def reg_log(string):
  # Open a file with access mode 'a'
  file_object = open('log.txt', 'a', encoding='utf-8')
  # Append 'DATE & TIME' and 'string' at the end of file
  file_object.write(f'\n{DATE} at {TIME}\n{string}\n\n')
  # Close the file
  file_object.close()

In [23]:
try:
  # start streaming video from webcam
  video_stream()
  # label for video
  label_html = 'Capturing...'
  # initialze bounding box to empty

  #path = 'best.pt' # number plate detection model
  #model = torch.hub.load('ultralytics/yolov5', 'custom', path, force_reload=True)

  bbox = ''
  count = 0 
  while True:
      js_reply = video_frame(label_html, 'bbox')
      if not js_reply:
          break
      else:
        try:
          # convert JS response to OpenCV Image/Frame
          frame = js_to_image(js_reply["img"]) 
          #results = model(frame)
          #reading image ocr to
          reader = easyocr.Reader(['bn'], gpu=True)
          result = reader.readtext(frame)

          #frame = np.squeeze(results.render())
          
          text = ''
          for string in result:
            text = text+string[1]+'\n'
          print(text)
          #reg_log(text) #writing time, entry, numberplate on txt file

          # Codes bellow under developement on rendering on image frame
          #fontpath = "banglamn.ttc"  # bangla font
          #font = ImageFont.truetype(fontpath, 24)
          #img_pil = Image.fromarray(frame)
          #draw = ImageDraw.Draw(img_pil)
          #b, g, r, a = 221, 82, 6, 0
          #draw.text((50, 80),  text, font=font, fill=(b, g, r, a))
          #frame = np.array(img_pil)

          #ret, buffer = cv2.imencode('.jpg', frame)
          #frame = buffer.tobytes()
        except:
          print('No source found')
          break
except:
  print('Something went wrong')

<IPython.core.display.Javascript object>

ঢাকা মেট্রো গ
৩৫-৩১০২

ঢাকা মেট্রেোগ
(
৩১০২

ঢাকা মেট্রেোগ
৩৫-৩১০২



3_9
799

_9 80

৭80


183
"1

ঢাকা মেেটো
১ 8-

ঢাকা েটো
8=0

ঢাকা মেটে
১ 8=

ঢাকা মেেট্রো:
8 - 0

ঢাকা মেেটরো:
১ 8= 0

ঢাকা মেেট্রো
১ 8-


>
৬-২৫৯১

২৬-২৫৯১

>
৬-২৫৯২

২৬-২৫৯২

৩ ২৮৯৭

চনদ6   রম_ডা
৩- ২৮৯৭

9 9- ২৮৯৭

৩-২৮৯৭

ক7 ম_োো:
৩-২৮৯৭

ঢালযা
মা_তাো:
৩-১৮৯৭

ম_রো:
9-২৮৯৭

ঢক ম_টো:
৩-২৮৯৭

ঢাক মতা:
99-২৮৯৭


ঢাকা মেট্রো গ
২৯-১৩৯২

ঢাকা মেট্রোগ
২৯-১৩৯

ঢাকা মেট্রো-গ
২৯-১৩৯২

ঢাকা মেট্রো গ
২৯-১৩৯


ঢাকা মেদ্রেো:
২১-০৪৫৪

ঢাকা মেট্রো ঘ
২১-০৪৫৪

ঢাকা মেট্রো-ঘ
২১-০৪৫৪

ঢাকা মেট্রো-ঘ
২১-০৪৫৪

9

ঢাকা মেেট্রোব
১ ৫ -৫০৬8

ঢাকা মেেট্রোব
১৫-৫০৬8

ঢাকা মেেট্রোব
১৫-৫০৬8

ঢাকা মেেট্রোব
১৫-৫ ০৬8

ঢাকা মেেট্রো ব
৫ -৫০৬

ঢাকা মেেট্রোব
১৫-৫০৬8

ঢাকা মেেট্রোব
১৫ -৫০৬8

ঢাকা মেেট্রোব
৫ ২৫০৬8


ঢাকা মেট্রো-ঘ
২১-১৫৮১
$৫ শহ5ড

ঢাকা মেট্রো-ঘ
২১-১৫৮১
$৫শশহড

ঢাকা মেট্রো-ঘ
২১-১৫৮১
$৫শ শহঞহ

ঢাকা মেট্রো-ঘ
২১-১৫৮১
$৫শ_শহড

1638
১৯৪-১:
98৮

' 33001
৬8৮

ঢাক} টে'
[$8
৬৪৮

ঢা৫
8
৬৪৮


গগসেেদ্ /
>০-0৫9৬

চাব বেন্রো-গ
9-৩৫৬

চাব বেন্রো-গ
9-9৫9